<h4>
    <b>
        Questão 3
    </b>
</h4>

In [4]:
import numpy as np 

def hilbert(m,n):

    H = np.zeros((m,n), dtype=float)

    for i in range(0,m):

        for j in range(0,n):

            H[i,j] = 1/(i+j+1)
            
    return H

def subs_reg(A):

    m, n = A.shape

    x = np.zeros(m)
    
    x[m-1] = A[m-1,-1] / A[m-1,-2]

    for i in range(m-2,-1,-1):
    
        soma = 0
        
        for j in range(i+1,n-1):
            
            soma = soma + A[i,j] * x[j]
        
        x[i] = (A[i,-1] - soma) / A[i,i]

    return x
    
def gauss_sem_pivotacao(A):
    
    m   = A.shape[0]
    M   = np.zeros_like(A, dtype=float)

    for k in range(0, m - 1):
        
        for i in range(k + 1, m):
            M[i, k] = A[i, k] / A[k, k]  
            A[i, :] = A[i, :] - M[i, k] * A[k, :]  
                
    return A


def piv_parc(A, k):

    i = np.argmax(np.abs(A[k:, k]))
    
    A[[k, i+k], :] = A[[i+k, k], :]

    return A

def gauss(A):
    
    m   = A.shape[0]
    M   = np.zeros_like(A, dtype=float)

    for k in range(0, m - 1):
        
        A = piv_parc(A, k) 
        
        if A[k, k] != 0: 
            for i in range(k + 1, m):
                M[i, k] = A[i, k] / A[k, k]  
                A[i, :] = A[i, :] - M[i, k] * A[k, :]  

    return A

def subs_dir(A):

    m, n = A.shape

    x = np.zeros(m)

    x[0] = A[0,-1] / A[0,0]
    
    for i in range(1,m):
        
        soma = 0
        
        for j in range(0,i+1):
            
            soma = soma + A[i,j] * x[j]
        
        x[i] = (A[i,-1] - soma) / A[i,i]

    return x
    
def lu(A):

    n = len(A)
    L = np.eye(n,n)
    
    for k in range(0,n-1):

        if A[k,k] == 0:
                return [],[]
        
        for i in range(k+1,n):
                    
            L[i,k] = A[i,k] / A[k,k]
            A[i,:] = A[i,:] - L[i,k] * A[k,:]
    
    return L, A 
    
# (A) ======================================================
H = hilbert(6,6)


# (b) =====================================================

# Obtendo H^{-1}
H_inv = np.linalg.inv(H.copy())

# calculando número de condicionamento usando || . ||_2
k = np.linalg.norm(H) * np.linalg.norm(H_inv)
print('cond',k)
print('O número de condicionamento de H k(H)={:.2e}'.format(k))

# (c) ======================================================
# Declarando vetor b
b = np.array([1, 1, 1, 1, 1, 1])
print('Vetor b=',b)

# (c.1) ====================================================
H = hilbert(6,6)
# Construindo a matriz [H|b]
Hb = np.column_stack((H,b))

#Obtém triangular superior
Ht = gauss_sem_pivotacao(Hb.copy())

#Obtém uma aproximação de x tal que Hx=b:
x_gauss_sem_piv = subs_reg(Ht.copy())

print('A solução encontrada pela eliminação de Gauss sem pivoteamento é: \n',x_gauss_sem_piv)
# Vejamos que é solução
print(H @ x_gauss_sem_piv)


# (c.2) ====================================================
H = hilbert(6,6)
# A matriz [H|b] já foi construida em (c.1).

# Obtém triangular superior com pivotacao parcial.
Htp = gauss(Hb.copy())

# Obtém x tal que Hx=b pela eliminação de gauss com pivotamento parcial. 
x_gauss_piv = subs_reg(Htp)

print('A solução encontrada pela eliminação de Gauss com pivoteamento parcial é: \n',x_gauss_piv)
# Vejamos que é solução
print(H @ x_gauss_piv)

# (c.3) =====================================================
H = hilbert(6,6)
#Obtendo a fatoração LU de H^{(6)}:
L, U = lu(H)

# Ly = b 
yl = subs_dir(np.column_stack((L,b)))

xl = subs_reg(np.column_stack((U,yl)))

print('Solução por Fatoração de LU x=\n',xl)

#Vejamos que é solução
print(L @ U @ xl)
H = hilbert(6,6)
# (c.4) ========================================================

# Verificando se é simétrica.
print(H == H.T)
# H é simétrica
def gaxpy_cholesky(A):

    n = len(A)
    
    for j in range(0,n):
        
        if j > 0:

            A[j:n,j] = A[j:n,j] - A[j:n,0:j] @ A[j,0:j].T
        
        A[j:n,j] = A[j:n,j] / np.sqrt(A[j,j])
        
    return np.tril(A)

G = gaxpy_cholesky(H)

#Gy=b
yg = subs_dir(np.column_stack((G,b)))

#G^t * x = y
xg = subs_reg(np.column_stack((G.T,yg)))

print('A solução por Cholesky é: xg=\n',xg)

print(b)

cond 15118987.128721727
O número de condicionamento de H k(H)=1.51e+07
Vetor b= [1 1 1 1 1 1]
A solução encontrada pela eliminação de Gauss sem pivoteamento é: 
 [-6.000e+00  2.100e+02 -1.680e+03  5.040e+03 -6.300e+03  2.772e+03]
[1. 1. 1. 1. 1. 1.]
A solução encontrada pela eliminação de Gauss com pivoteamento parcial é: 
 [-6.000e+00  2.100e+02 -1.680e+03  5.040e+03 -6.300e+03  2.772e+03]
[1. 1. 1. 1. 1. 1.]
Solução por Fatoração de LU x=
 [-6.000e+00  2.100e+02 -1.680e+03  5.040e+03 -6.300e+03  2.772e+03]
[1. 1. 1. 1. 1. 1.]
[[ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]
 [ True  True  True  True  True  True]]
A solução por Cholesky é: xg=
 [-6.000e+00  2.100e+02 -1.680e+03  5.040e+03 -6.300e+03  2.772e+03]
[1 1 1 1 1 1]
